In [9]:
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [10]:
base_dir = 'D:/AI/tester01'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# Parameters
img_size = 299 
batch_size = 32
split_ratio = 0.2 

In [11]:
def split_data(source, train, val, test, split_size):
    files = os.listdir(source)
    shuffled_files = np.random.permutation(files)

    val_count = int(len(files) * split_size)
    test_count = val_count
    train_count = len(files) - val_count - test_count

    train_files = shuffled_files[:train_count]
    val_files = shuffled_files[train_count:train_count+val_count]
    test_files = shuffled_files[train_count+val_count:]

    for category in [(train_files, train), (val_files, val), (test_files, test)]:
        for file in category[0]:
            shutil.copy(os.path.join(source, file), category[1])

# Create directories
for folder in ['train', 'val', 'test']:
    for category in ['normal', 'cataract', 'surgery']:
        os.makedirs(os.path.join(base_dir, folder, category), exist_ok=True)

# Split the dataset
for category in ['normal', 'cataract', 'surgery']:
    split_data(os.path.join(base_dir, category), 
               os.path.join(train_dir, category),
               os.path.join(val_dir, category),
               os.path.join(test_dir, category),
               split_ratio)

In [12]:
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical')

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical')

Found 1842 images belonging to 3 classes.
Found 611 images belonging to 3 classes.
Found 611 images belonging to 3 classes.


In [13]:
base_model = InceptionV3(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet')

# Freeze the base_model
base_model.trainable = False

# Add custom layers
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
predictions = layers.Dense(3, activation='softmax')(x) 

model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

In [14]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.n // batch_size)

Epoch 1/10
57/57 [==============================] - 97s 2s/step - loss: 1.7709 - accuracy: 0.6591 - val_loss: 1.7921 - val_accuracy: 0.5872
Epoch 2/10
57/57 [==============================] - 90s 2s/step - loss: 0.5077 - accuracy: 0.8099 - val_loss: 0.2471 - val_accuracy: 0.9095
Epoch 3/10
57/57 [==============================] - 90s 2s/step - loss: 0.3889 - accuracy: 0.8569 - val_loss: 0.1757 - val_accuracy: 0.9326
Epoch 4/10
57/57 [==============================] - 90s 2s/step - loss: 0.2727 - accuracy: 0.8978 - val_loss: 0.2775 - val_accuracy: 0.8865
Epoch 5/10
57/57 [==============================] - 90s 2s/step - loss: 0.2705 - accuracy: 0.9033 - val_loss: 0.1517 - val_accuracy: 0.9391
Epoch 6/10
57/57 [==============================] - 90s 2s/step - loss: 0.2447 - accuracy: 0.9133 - val_loss: 0.1560 - val_accuracy: 0.9457
Epoch 7/10
57/57 [==============================] - 91s 2s/step - loss: 0.2010 - accuracy: 0.9249 - val_loss: 0.1641 - val_accuracy: 0.9359
Epoch 8/10
57/57 [==

In [15]:
eval_result = model.evaluate(test_generator)
print(f"Test Loss: {eval_result[0]}, Test Accuracy: {eval_result[1]}")

20/20 [==============================] - 24s 1s/step - loss: 0.2139 - accuracy: 0.9329
Test Loss: 0.2138526737689972, Test Accuracy: 0.9328969120979309


In [17]:
model.save('cataract.h5')